In [1]:
from z3 import *

def compute_tax(category, subcategory, quantity, alcohol_content=None):
    """
    計算菸酒稅的應納稅額
    參數:
      category: '菸' 或 '酒'
      subcategory: 
         若 category 為 '菸'，則必須為 1~8（依題目定義）
           1. 紙菸（新制，106年6月11日以後）：每千支 1590元
           2. 菸絲（新制）：每公斤 1590元
           3. 雪茄（新制）：每公斤 1590元
           4. 其他菸品（新制）：每公斤 1590元  （※行政院112.4.1規定，取每公斤或每千支之較高者，這裡簡化取 1590元）
           5. 紙菸（舊制，106年6月11日以前）：每千支 590元   【依 菸酒稅法第7條】
           6. 菸絲（舊制）：每公斤 590元                         【依 菸酒稅法第7條】
           7. 雪茄（舊制）：每公斤 590元                         【依 菸酒稅法第7條】
           8. 其他菸品（舊制）：每公斤 590元                       【依 菸酒稅法第7條】
         若 category 為 '酒'，則必須為 1~10（依題目定義）
           1. 釀造酒類：啤酒，每公升 26元                        【依 酒類規定】
           2. 釀造酒類：其他釀造酒，每公升按酒精成分每度 7元       【依 酒類規定】
           3. 蒸餾酒類：每公升按酒精成分每度 2.5元                  【依 酒類規定】
           4. 再製酒類：酒精成分超過20%者，每公升 185元             【依 酒類規定】
           5. 再製酒類：酒精成分20%或以下者，每公升按酒精成分每度 7元 【依 酒類規定】
           6. 料理酒類：每公升 9元                                【依 酒類規定】
           7. 料理酒類：97年5月15日以前出廠者，每公升 22元          【依 酒類規定】
           8. 其他酒類：每公升按酒精成分每度 7元                   【依 酒類規定】
           9. 酒精：每公升 15元                                  【依 酒類規定】
          10. 酒精：97年5月15日以前出廠者，每公升 11元            【依 酒類規定】
      quantity: 當月份出廠應稅數量（單位依子類別而定：菸類為「支」或「公斤」，酒類為「公升」）
      alcohol_content: 酒精成分（度數）；若為酒類且有依酒精成分計算，該參數必須提供
  
    return:
      應納稅額（元）
    """
    # 建立 Z3 求解器
    s = Solver()
    tax = Real('tax')  # 最終稅額

    if category == '菸':
        # 菸類另外徵收健康福利捐：
        # 紙菸、菸絲、雪茄、其他菸品均為每單位（千支或公斤）徵收 1000元
        if subcategory in [1, 5]:  # 紙菸：依生產日期區分新制/舊制
            # 單位：每千支
            # 新制（subcat 1）：基本稅率 1590元【每千支徵收1,590元】
            # 舊制（subcat 5）：基本稅率 590元  【依 菸酒稅法第7條】
            base_rate = 1590 if subcategory == 1 else 590
            health_rate = 1000  # 【菸品健康福利捐規定】
            unit_divisor = 1000  # 計算單位為千支
        elif subcategory in [2, 6]:  # 菸絲：單位為公斤
            base_rate = 1590 if subcategory == 2 else 590  # 【依 菸酒稅法第7條】
            health_rate = 1000  # 【菸品健康福利捐規定】
            unit_divisor = 1
        elif subcategory in [3, 7]:  # 雪茄：單位為公斤
            base_rate = 1590 if subcategory == 3 else 590  # 【依 菸酒稅法第7條】
            health_rate = 1000
            unit_divisor = 1
        elif subcategory in [4, 8]:  # 其他菸品：單位為公斤
            base_rate = 1590 if subcategory == 4 else 590  # 【依 菸酒稅法第7條】
            health_rate = 1000
            unit_divisor = 1
        else:
            raise ValueError("菸類子類別需為1至8中的數字。")
        
        # 計算公式： (基本稅率 + 健康福利捐稅率) * (出廠數量 / 單位)
        tax_expr = (base_rate + health_rate) * (quantity / unit_divisor)
        s.add(tax == tax_expr)
    
    elif category == '酒':
        # 酒類：必須提供 alcohol_content（酒精成分，單位：度）
        if alcohol_content is None:
            raise ValueError("酒類計算時，必須提供酒精成分(度數)")
        # 使用 Z3 的 If 條件式依子類別選擇對應的稅率：
        rate = If(subcategory == 1, 26,                      # 釀造酒類：啤酒 每公升26元 【依 酒類規定】
              If(subcategory == 2, 7 * alcohol_content,      # 釀造酒類：其他釀造酒 每公升按酒精成分每度7元 【依 酒類規定】
              If(subcategory == 3, 2.5 * alcohol_content,      # 蒸餾酒類：每公升按酒精成分每度2.5元 【依 酒類規定】
              If(subcategory == 4, 185,                        # 再製酒類：酒精成分超過20%者 每公升185元 【依 酒類規定】
              If(subcategory == 5, 7 * alcohol_content,        # 再製酒類：酒精成分20%或以下者 每公升按酒精成分每度7元 【依 酒類規定】
              If(subcategory == 6, 9,                          # 料理酒類：每公升9元 【依 酒類規定】
              If(subcategory == 7, 22,                         # 料理酒類：97年5月15日以前出廠者 每公升22元 【依 酒類規定】
              If(subcategory == 8, 7 * alcohol_content,        # 其他酒類：每公升按酒精成分每度7元 【依 酒類規定】
              If(subcategory == 9, 15,                         # 酒精：每公升15元 【依 酒類規定】
              If(subcategory == 10, 11,                        # 酒精：97年5月15日以前出廠者 每公升11元 【依 酒類規定】
              0))))))))))
        # 計算公式： 稅率 * 當月份出廠數量(公升)
        tax_expr = rate * quantity
        s.add(tax == tax_expr)
    
    else:
        raise ValueError("類別必須為 '菸' 或 '酒'")
    
    # 求解
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax)
    else:
        return None

#---------------------------------------------
# 測試範例

if __name__ == "__main__":
    # 範例1：菸類，假設使用新制 紙菸 (子類別 1)，當月份出廠 2000 支
    tax_tobacco = compute_tax(category='菸', subcategory=1, quantity=2000)
    print("菸類稅額 (新制紙菸, 2000支):", tax_tobacco)
    
    # 範例2：菸類，舊制 菸絲 (子類別 6)，當月份出廠 50 公斤
    tax_tobacco_old = compute_tax(category='菸', subcategory=6, quantity=50)
    print("菸類稅額 (舊制菸絲, 50公斤):", tax_tobacco_old)
    
    # 範例3：酒類，啤酒 (子類別 1)，當月份出廠 100 公升 (啤酒不需要酒精成分)
    tax_alcohol_beer = compute_tax(category='酒', subcategory=1, quantity=100, alcohol_content=0)
    print("酒類稅額 (啤酒, 100公升):", tax_alcohol_beer)
    
    # 範例4：酒類，其他釀造酒 (子類別 2)，酒精成分 5 度，當月份出廠 80 公升
    tax_alcohol_other = compute_tax(category='酒', subcategory=2, quantity=80, alcohol_content=5)
    print("酒類稅額 (其他釀造酒, 80公升, 酒精5度):", tax_alcohol_other)


菸類稅額 (新制紙菸, 2000支): 5180
菸類稅額 (舊制菸絲, 50公斤): 79500
酒類稅額 (啤酒, 100公升): 2600
酒類稅額 (其他釀造酒, 80公升, 酒精5度): 2800


In [10]:
import random
import time
import decimal
from decimal import Decimal, ROUND_HALF_UP
from fractions import Fraction
from z3 import Real, Solver, If, sat
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# ------------------ Z3 稅額計算模型 ------------------
def compute_tax(category, subcategory, quantity, alcohol_content=None):
    """
    計算菸酒稅的應納稅額
    參數:
      category: '菸' 或 '酒'
      subcategory: 
         若 category 為 '菸'，則必須為 1~8（依照題目定義）
           1. 紙菸（新制）：每千支徵收1,590元
           2. 菸絲（新制）：每公斤徵收1,590元
           3. 雪茄（新制）：每公斤徵收1,590元
           4. 其他菸品（新制）：每公斤徵收1,590元
           5. 紙菸（舊制）：每千支徵收590元  【依 菸酒稅法第7條】
           6. 菸絲（舊制）：每公斤徵收590元    【依 菸酒稅法第7條】
           7. 雪茄（舊制）：每公斤徵收590元    【依 菸酒稅法第7條】
           8. 其他菸品（舊制）：每公斤徵收590元  【依 菸酒稅法第7條】
         若 category 為 '酒'，則必須為 1~10（依照題目定義）
           1. 釀造酒類：啤酒，每公升26元
           2. 釀造酒類：其他釀造酒，每公升按酒精成分每度7元
           3. 蒸餾酒類：每公升按酒精成分每度2.5元
           4. 再製酒類：酒精成分超過20%者，每公升185元
           5. 再製酒類：酒精成分在20%以下者，每公升按酒精成分每度7元
           6. 料理酒類：每公升9元
           7. 料理酒類：97年5月15日以前出廠者，每公升22元
           8. 其他酒類：每公升按酒精成分每度7元
           9. 酒精：每公升15元
          10. 酒精：97年5月15日以前出廠者，每公升11元
      quantity: 當月份出廠應稅數量（菸類以「支」或「公斤」、酒類以「公升」）
      alcohol_content: 酒精成分（度數）；此參數始終提供，若不需則填入1
    return:
      應納稅額（整數，四捨五入後）
    """
    s = Solver()
    tax = Real('tax')
    
    if category == '菸':
        # 僅根據基本稅率計算（移除健康福利捐）
        if subcategory in [1, 5]:  # 紙菸，以每千支計算
            base_rate = 1590 if subcategory == 1 else 590
            unit_divisor = 1000
        elif subcategory in [2, 6]:  # 菸絲：每公斤
            base_rate = 1590 if subcategory == 2 else 590
            unit_divisor = 1
        elif subcategory in [3, 7]:  # 雪茄：每公斤
            base_rate = 1590 if subcategory == 3 else 590
            unit_divisor = 1
        elif subcategory in [4, 8]:  # 其他菸品：每公斤
            base_rate = 1590 if subcategory == 4 else 590
            unit_divisor = 1
        else:
            raise ValueError("菸類子類別需為1至8中的數字。")
        tax_expr = base_rate * (quantity / unit_divisor)
        s.add(tax == tax_expr)
    
    elif category == '酒':
        # 不論子類別是否需要酒精成分，皆假定提供一個值（不需的話填入1）
        if alcohol_content is None:
            alcohol_content = 1
        rate = If(subcategory == 1, 26,                      # 釀造酒類：啤酒，每公升26元
              If(subcategory == 2, 7 * alcohol_content,      # 釀造酒類：其他釀造酒，每度7元
              If(subcategory == 3, 2.5 * alcohol_content,      # 蒸餾酒類：每度2.5元
              If(subcategory == 4, 185,                        # 再製酒類：酒精成分超過20%者，每公升185元
              If(subcategory == 5, 7 * alcohol_content,        # 再製酒類：酒精成分在20%以下者，每度7元
              If(subcategory == 6, 9,                          # 料理酒類：每公升9元
              If(subcategory == 7, 22,                         # 料理酒類：97年5月15日以前出廠，每公升22元
              If(subcategory == 8, 7 * alcohol_content,        # 其他酒類：每度7元
              If(subcategory == 9, 15,                         # 酒精：每公升15元
              If(subcategory == 10, 11,                        # 酒精：97年5月15日以前出廠，每公升11元
              0))))))))))
        tax_expr = rate * quantity
        s.add(tax == tax_expr)
    else:
        raise ValueError("類別必須為 '菸' 或 '酒'")
    
    if s.check() == sat:
        m = s.model()
        tax_val = m.evaluate(tax)
        tax_str = tax_val.as_decimal(10).replace("?", "")
        tax_int = int(Decimal(tax_str).to_integral_value(rounding=ROUND_HALF_UP))
        return tax_int
    else:
        return None

# ------------------ Selenium 模擬逐字輸入函式 ------------------
def human_type(element, text, delay=0.1):
    """模擬人類逐字輸入 text，每個字間隔 delay 秒，最後送出 TAB"""
    for char in text:
        element.send_keys(char)
        time.sleep(delay)
    element.send_keys(Keys.TAB)

# ------------------ Selenium 測試流程 ------------------
def run_corporate_tax_test():
    driver = webdriver.Chrome()
    driver.delete_all_cookies()
    driver.get("https://www.etax.nat.gov.tw/etwmain/etw158w/75")
    
    total_tests = 0
    mismatch_tests = 0
    wait = WebDriverWait(driver, 10)
    
    for i in range(100):
        print(f"===== 迴圈第 {i+1} 次 =====")
        total_tests += 1
        
        category = random.choice(["菸", "酒"])
        
        if category == "菸":
            subcat = random.randint(1, 8)
            if subcat in [1, 5]:
                quantity = random.randint(1, 10) * 1000
            else:
                quantity = random.randint(1, 100)
            
            local_tax = compute_tax(category, subcat, quantity)
            
            # 填寫菸類表單
            sel_category = Select(wait.until(EC.presence_of_element_located((By.ID, "select1"))))
            sel_category.select_by_value("1")
            time.sleep(0.5)
            
            sel_subcat = Select(driver.find_element(By.ID, "select2"))
            sel_subcat.select_by_index(subcat)  # index 0 為「請選擇」
            
            input_quantity = driver.find_element(By.ID, "monthTax1")
            input_quantity.clear()
            human_type(input_quantity, str(quantity))
            
        else:  # 類別為酒
            subcat = random.randint(1, 10)
            quantity = random.randint(1, 100)
            # 對酒類，無論子類別如何，皆填寫酒精成分欄位
            if subcat in [2, 3, 5, 8]:
                alcohol_content = random.randint(1, 40)  # 避免0
            else:
                alcohol_content = 1  # 預設填入1
            try:
                input_degree = driver.find_element(By.ID, "degree")
                input_degree.clear()
                human_type(input_degree, str(alcohol_content))
            except Exception:
                print("找不到酒精成分輸入框，預設使用1度")
                alcohol_content = 1
            
            local_tax = compute_tax(category, subcat, quantity, alcohol_content)
            
            # 填寫酒類表單
            sel_category = Select(wait.until(EC.presence_of_element_located((By.ID, "select1"))))
            sel_category.select_by_value("2")
            time.sleep(0.5)
            
            sel_subcat = Select(driver.find_element(By.ID, "select3"))
            sel_subcat.select_by_index(subcat)  # index 0 為「請選擇」
            
            # 無論如何，月出廠數量欄位必填
            input_quantity = driver.find_element(By.ID, "monthTax2")
            input_quantity.clear()
            human_type(input_quantity, str(quantity))
        
        # 點擊「計算」按鈕
        calc_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@title='計算']")))
        calc_button.click()
        
        time.sleep(2)
        
        result_elem = driver.find_element(By.CSS_SELECTOR, "span.text-danger")
        online_result_text = result_elem.text.strip()
        try:
            online_tax = int(Decimal(online_result_text.replace(',', '')).to_integral_value(rounding=ROUND_HALF_UP))
        except Exception as e:
            online_tax = None
            print("線上結果解析錯誤:", online_result_text)
        
        if online_tax is None or local_tax != online_tax:
            mismatch_tests += 1
            print(">>> 不一致參數發現!")
            print(f"Category: {category}")
            if category == "菸":
                print(f"子類別: {subcat}, 數量: {quantity}")
            else:
                print(f"子類別: {subcat}, 酒精成分: {alcohol_content}, 數量: {quantity}")
            print(f"本地計算稅額: {local_tax}, 線上稅額: {online_tax}")
        else:
            print("比對一致。")
        
        driver.refresh()
        time.sleep(2)
    
    print(f"總測試次數: {total_tests}, 不一致次數: {mismatch_tests}")
    driver.quit()

if __name__ == "__main__":
    run_corporate_tax_test()


===== 迴圈第 1 次 =====
比對一致。
===== 迴圈第 2 次 =====
找不到酒精成分輸入框，預設使用1度
比對一致。
===== 迴圈第 3 次 =====
比對一致。
===== 迴圈第 4 次 =====
比對一致。
===== 迴圈第 5 次 =====
比對一致。
===== 迴圈第 6 次 =====
找不到酒精成分輸入框，預設使用1度
比對一致。
===== 迴圈第 7 次 =====
找不到酒精成分輸入框，預設使用1度
>>> 不一致參數發現!
Category: 酒
子類別: 3, 酒精成分: 1, 數量: 30
本地計算稅額: 75, 線上稅額: 0
===== 迴圈第 8 次 =====
比對一致。
===== 迴圈第 9 次 =====
比對一致。
===== 迴圈第 10 次 =====
比對一致。
===== 迴圈第 11 次 =====
比對一致。
===== 迴圈第 12 次 =====
找不到酒精成分輸入框，預設使用1度
比對一致。
===== 迴圈第 13 次 =====
比對一致。
===== 迴圈第 14 次 =====
比對一致。
===== 迴圈第 15 次 =====
比對一致。
===== 迴圈第 16 次 =====
找不到酒精成分輸入框，預設使用1度
比對一致。
===== 迴圈第 17 次 =====
找不到酒精成分輸入框，預設使用1度
比對一致。
===== 迴圈第 18 次 =====
比對一致。
===== 迴圈第 19 次 =====
找不到酒精成分輸入框，預設使用1度
比對一致。
===== 迴圈第 20 次 =====
找不到酒精成分輸入框，預設使用1度
比對一致。
===== 迴圈第 21 次 =====
比對一致。
===== 迴圈第 22 次 =====
找不到酒精成分輸入框，預設使用1度
比對一致。
===== 迴圈第 23 次 =====
比對一致。
===== 迴圈第 24 次 =====
找不到酒精成分輸入框，預設使用1度
>>> 不一致參數發現!
Category: 酒
子類別: 3, 酒精成分: 1, 數量: 48
本地計算稅額: 120, 線上稅額: 0
===== 迴圈第 25 次 =====
找不到酒精成分輸入框，預設使用1度
>>> 不一致參數發現!
Ca